# <font color=blue>  Mapa de Calor  </font>
## <font color=black>  Distribuição de IRA dos alunos de Engenharia da Computação pelo mapa do DF </font>
#### <font color=black> Escopo de análise delimitado entre o primeiro semestre de 2009 até o último semestre de 2018 do curso de Engenharia da Computação da Universidade de Brasília.</font>

In [1]:
#importar bibliotecas

import numpy as np
import pandas as pd
from geopy.geocoders import GoogleV3
import csv
from geopy import geocoders
import json
from pathlib import Path
import pycep_correios
import warnings
import folium
from folium.plugins import HeatMap
import os
import subprocess
import time
import re
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import plotly
import plotly.graph_objs as go
import plotly.plotly as py
import cufflinks

#parametros para rodar API do Google
g = geocoders.GoogleV3(api_key='chave API do Goohgle Maps')
geolocator = GoogleV3()

#ignorar avisos do código
warnings.filterwarnings('ignore')




/Users/air/Documents/repositories/estatisticas_departamento/venv/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning:

Since July 2018 Google requires each request to have an API key. Pass a valid `api_key` to GoogleV3 geocoder to hide this warning. See https://developers.google.com/maps/documentation/geocoding/usage-and-billing



In [7]:
path = 'data/anoecep_ira_parte15.csv'
df_cep =  pd.read_csv(path, sep =';')
df_cep = df_cep.fillna(0)
df_cep.head()

,Semestre/Ano,CEP,0,0.0333,0.0625,0.08,0.1473,0.1667,0.1778,0.1818,...,4.6969,4.7006,4.7425,4.75,4.7647,4.8244,4.8379,4.8621,5,Aluno sem IRA (Calouro)
0,1996/1,71699016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1996/2,71699016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1997/1,71699016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1997/2,71699016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1998/1,71699016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### <font color=black> Coleta dos bairros que se encontram no escopo analisado</font>

In [ ]:
bairro = []
bairro2 = []
for index, row in df_cep.iterrows():
    try:
        endereco = pycep_correios.consultar_cep(str(row.CEP))
    except:
        try:
            endereco['bairro'] = 'NaN'
            location = g.geocode(row.CEP, timeout=10)
            location = str(location)
            endereco['bairro'] = location.partition(" -")[0]
            endereco['bairro'] = location.partition(" ,")[0]
            endereco['bairro'] = location.partition(",")[0]
            print(row.CEP)
            print("google " + endereco['bairro'])
            
        except:
            endereco['bairro'] = 'NaN'
            print(row.CEP)
            
    if endereco['bairro'] is None:
        bairro2.append('NaN')
        continue
    try:
        if  '(' in endereco['bairro']:     
            s = endereco['bairro']
            temp = s[s.find("(")+1:s.find(")")]
        else:
            temp = endereco['bairro']
    except:
        print('erro' + str(endereco['bairro']))
    if not(temp in bairro):
        print(temp)
        bairro.append(temp)
        row.CEP
        bairro.append(endereco['bairro'])
    bairro2.append(temp)
    

São Sebastião
Guará
Ceilândia
Asa Sul
Paranoá
Setor de Habitações Individuais Norte
Candangolândia
Sobradinho
70000000
NaN
70000000
Área Octogonal
Asa Norte
70000000
Núcleo Bandeirante
73380100
70000000
73380100
Jóquei
70000000
Cruzeiro Velho
Guará II
Setor de Habitações Individuais Sul
72900000
Planaltina
73380100
70000000
Setor Sudoeste
Gama
Recanto das Emas
72880000
72900000
73380100
Alvorada
70000000
Cruzeiro Novo
72110600
72333210
72880000
72900000
Ahú
70000000
Guará I
72110600
72333210
72880000
72900000
73380400
70000000
72110600
72333210
73380400
4119010
70000000
72110600
Taguatinga
72333210
72900000
73380400
Setor Noroeste
Águas Claras
72110600
72333210
72900000
73370100
73380400
Vila Redenção
Setor Habitacional Jardim Botânico
Riacho Fundo I
72110600
72110800
Samambaia
72333210
Santa Maria
72900000
Setor de Mansões de Sobradinho
73370100
73380400
São Cristóvão
43330000
Vila Estrutural
Varjão
Park Way
72110600
72110800
72333210
72900000
73370100
43330000
72110600
72110800
73370

#### <font color=black> Captura dos dados da Latitude e Longitude dos bairros Analisados</font>

In [ ]:
my_file = Path("data/cep_bairro.json")

if my_file.is_file():
    bairro_cep = json.load(open('data/cep_bairro.json'))
else:
    bairro_cep ={}
    with open("data/cep_uni.txt", "r") as f:
        f = f.read().splitlines() 
        for x in f:

            if not(str(x) in bairro_cep):
                
                inputAddress = str(x)
                location = g.geocode(inputAddress, timeout=10)
                location = str(location)

                if location == 'None':
                    try:
                        endereco = pycep_correios.consultar_cep(x)
                        inputAddress = endereco['end']+ " , " + endereco['cidade']
                        location = g.geocode(inputAddress, timeout=10)
                        location = str(location)
                    
                        if location is None:
                            bairro_cep[x] = "NaN"
                        else:
                            bairro_cep[x] = location

                    except:
                        bairro_cep[x] = "NaN"
                
                else:
                    bairro_cep[x] = location

    with open('data/cep_bairro.json', 'w') as file:
        json.dump(bairro_cep, file)

 

In [ ]:
lat_log = json.load(open('data/lat_log_uni.json'))
df_cep['CEP'] = df_cep['CEP'].astype(str)
df_cep['bairro'] = df_cep['CEP']
df_cep = df_cep.replace({"CEP": lat_log})
df_cep = df_cep.replace({"bairro": bairro_cep})

df_cep.head()

In [ ]:
path = 'data/bairro_cep.csv'
df_cep =  pd.read_csv(path, sep =';')
df_cep = df_cep.fillna(0)
df_cep = df_cep.replace({"CEP": lat_log})
df_cep.head()

In [50]:
my_file = Path("data/bairro_lat_long_uni.json")

if my_file.is_file():
    bairro_lat = json.load(open('data/bairro_lat_long_uni.json'))
else:
    bairro_lat ={}
    with open("data/bairros.txt", "r") as f:
        f = f.read().splitlines() 
        for x in f:

            if not(str(x) in bairro_lat):
                
                inputAddress = str(x) + ", Brasilia"
                location = g.geocode(inputAddress, timeout=10)

                if location is None:
                    try:
                        endereco = pycep_correios.consultar_cep(x)
                        inputAddress = endereco['end']+ " , " + endereco['cidade']
                        location = g.geocode(inputAddress, timeout=10)
                    
                        if location is None:
                            bairro_lat[x] = "NaN"
                        else:
                            bairro_lat[x] = location

                    except:
                        bairro_lat[x] = "NaN"
                
                else:
                    bairro_lat[x] = str(location.latitude)+ "/"+ str(location.longitude)
    with open('data/bairro_lat_long_uni.json', 'w') as file:
        json.dump(bairro_lat, file)

 

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [67]:
df_cep['bairro2'] = df_cep['bairro']
df_cep = df_cep.replace({"bairro2": bairro_lat})
df_cep.head()

,Unnamed: 0,Semestre/Ano,CEP,0,0.0333,0.0625,0.08,0.1473,0.1667,0.1778,...,4.7425,4.75,4.7647,4.8244,4.8379,4.8621,5,Aluno sem IRA (Calouro),bairro,bairro2
0,0,1996/1,0.332538,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Jardins Mangueiral,-15.8940497/-47.8007606
1,1,1996/2,0.332538,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Jardins Mangueiral,-15.8940497/-47.8007606
2,2,1997/1,0.332538,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Jardins Mangueiral,-15.8940497/-47.8007606
3,3,1997/2,0.332538,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Jardins Mangueiral,-15.8940497/-47.8007606
4,4,1998/1,0.332538,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Jardins Mangueiral,-15.8940497/-47.8007606


In [54]:
df_cep.to_csv('data/lat_ira.csv', sep=',')

In [99]:
#Agrupar os valores importantes para a criação do mapa de calor
#ano = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
#ano de 2019 foge do escopo, apesar de poder ser incluido na análise

ano = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]
semestre = [0,1,2]
time_dict = {}
for x in ano:
    for y in semestre:
        time_dict[str(x)+'_'+str(y)] = []
        df = df_cep.loc[df_cep['Semestre/Ano'] == str(x)+"/"+str(y)]
        lista = df.index.values.astype(int)
        arr = df.drop(['Unnamed: 0','Semestre/Ano', 'CEP', 'bairro','bairro2'], axis=1).values
        index_names = df.drop(['Unnamed: 0','Semestre/Ano','CEP', 'bairro','bairro2'], axis=1).index
        col_names = df.drop(['Unnamed: 0','Semestre/Ano','CEP', 'bairro','bairro2'], axis=1).columns
        R,C = np.where(arr>0)
        out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
        df_out = pd.DataFrame(data = out_arr)
        for i in lista:
            for j in out_arr:
                if j[0] == i:
                    time_dict[str(x)+'_'+str(y)].append([df.bairro[i],df.bairro2[i],float(j[1])])


In [113]:
bairro_ira_dict = {}
ano = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
semestre = [0,1,2]
for x in ano:
    for y in semestre:
        bairro_ira_dict[str(x)+'_'+str(y)] = []
        i = time_dict[str(x)+'_'+str(y)]
        df = pd.DataFrame(i,columns = ['bairro','lat_long','ira'])
        a = pd.Series({c: df[c].unique() for c in df})
        for ele in a[0]:
            if ((ele == 0) or(ele == '0')):
                df.bairro[df['bairro'] == ele] = "Nao Informado"
                continue
            if ele == "Nao Informado":
                df.lat_long[df['bairro'] == ele][0] = "-15.6886495/-47.9772722"
                
            lat = df.lat_long[df['bairro'] == ele].values[0]
            media = df.ira[df['bairro'] == ele].mean()
            bairro_ira_dict[str(x)+'_'+str(y)].append([ele,lat,media])

#### <font color=black> Criação dos mapas de calor</font>

In [ ]:
pathto ='/heatmap_ira/'
options = Options()
#options.headless = True

for i in range(33):
    fn='testmap.html'
    tmpurl= pathto + 'heatmap'+str(i)+'.html'.format(path=os.getcwd(),mapfile=fn)

    browser = webdriver.Firefox()
    browser.get(tmpurl)
    time.sleep(20)  #Give the map tiles some time to load
    browser.save_screenshot('map'+str(i)+'.png')
    browser.quit()

In [ ]:
a = [26]
for i in a:
    fn='testmap.html'
    tmpurl= pathto+'heatmap'+str(i)+'.html'.format(path=os.getcwd(),mapfile=fn)


    browser = webdriver.Firefox(options=options)
    browser.get(tmpurl)
    time.sleep(20)  #Give the map tiles some time to load
    browser.save_screenshot('map'+str(i)+'.png')
    browser.quit()